# 05898 Data Science HW4 
# Customer Segmentation using K-means clustering
## Xin Qiu   Andrew ID: xq

## Step 1: Data preprocessing

In [438]:
# You can use pandas to preprocess the dataset and sklearn package to perform the KMeans Clustering Analysis
import pandas as pd
from sklearn.cluster import KMeans

In [439]:
# Loading all data into individual data frames
customers = pd.read_excel("bikeshops.xlsx",sheetIndex=1)
orders = pd.read_excel("orders.xlsx",sheetIndex=1)
products = pd.read_excel("bikes.xlsx",sheetIndex=1)

In [440]:
# Combining above data into a single dataframe
ordersXcustomers = pd.merge(orders,customers,left_on = "customer.id",right_on = "bikeshop.id")
ordersXcustomersXproducts = pd.merge(ordersXcustomers, products, left_on = "product.id",right_on = "bike.id")
ordersXcustomersXproducts.head()

order.id  order.line order.date  customer.id  product.id  quantity  \
0         1           1 2011-01-07            2          48         1   
1       132           6 2011-05-13            2          48         1   
2       507           2 2012-06-26            2          48         1   
3       528          18 2012-07-16            2          48         1   
4       691          13 2013-02-05            2          48         1   

   bikeshop.id             bikeshop.name bikeshop.city bikeshop.state  \
0            2  Ithaca Mountain Climbers        Ithaca             NY   
1            2  Ithaca Mountain Climbers        Ithaca             NY   
2            2  Ithaca Mountain Climbers        Ithaca             NY   
3            2  Ithaca Mountain Climbers        Ithaca             NY   
4            2  Ithaca Mountain Climbers        Ithaca             NY   

    latitude  longitude  bike.id            model category1      category2  \
0  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   
1  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   
2  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   
3  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   
4  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   

    frame  price  
0  Carbon   6070  
1  Carbon   6070  
2  Carbon   6070  
3  Carbon   6070  
4  Carbon   6070

In [441]:
# Categorizing prices column into 2 bins: High and Low
import numpy as np
medianprice = np.median(ordersXcustomersXproducts['price'])
ordersXcustomersXproducts['Price Category'] = ordersXcustomersXproducts.apply(lambda row: 'High' if row.price > medianprice else 'Low', axis=1)
ordersXcustomersXproducts.head()

order.id  order.line order.date  customer.id  product.id  quantity  \
0         1           1 2011-01-07            2          48         1   
1       132           6 2011-05-13            2          48         1   
2       507           2 2012-06-26            2          48         1   
3       528          18 2012-07-16            2          48         1   
4       691          13 2013-02-05            2          48         1   

   bikeshop.id             bikeshop.name bikeshop.city bikeshop.state  \
0            2  Ithaca Mountain Climbers        Ithaca             NY   
1            2  Ithaca Mountain Climbers        Ithaca             NY   
2            2  Ithaca Mountain Climbers        Ithaca             NY   
3            2  Ithaca Mountain Climbers        Ithaca             NY   
4            2  Ithaca Mountain Climbers        Ithaca             NY   

    latitude  longitude  bike.id            model category1      category2  \
0  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   
1  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   
2  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   
3  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   
4  42.443961 -76.501881       48  Jekyll Carbon 2  Mountain  Over Mountain   

    frame  price Price Category  
0  Carbon   6070           High  
1  Carbon   6070           High  
2  Carbon   6070           High  
3  Carbon   6070           High  
4  Carbon   6070           High

In [442]:
# Drop columns unnecessary for clustering
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('latitude', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('longitude', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('customer.id', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('product.id', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('bikeshop.city', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('bikeshop.state', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('bikeshop.id', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('bike.id', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('order.id', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('order.line', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('price', 1)
ordersXcustomersXproducts = ordersXcustomersXproducts.drop('order.date', 1)
ordersXcustomersXproducts.head()

quantity             bikeshop.name            model category1  \
0         1  Ithaca Mountain Climbers  Jekyll Carbon 2  Mountain   
1         1  Ithaca Mountain Climbers  Jekyll Carbon 2  Mountain   
2         1  Ithaca Mountain Climbers  Jekyll Carbon 2  Mountain   
3         1  Ithaca Mountain Climbers  Jekyll Carbon 2  Mountain   
4         1  Ithaca Mountain Climbers  Jekyll Carbon 2  Mountain   

       category2   frame Price Category  
0  Over Mountain  Carbon           High  
1  Over Mountain  Carbon           High  
2  Over Mountain  Carbon           High  
3  Over Mountain  Carbon           High  
4  Over Mountain  Carbon           High

In [443]:
# To start, we’ll need a unit of measure to cluster on. We can select quantity purchased or total value of purchases. 
# We’ll select quantity purchased because total value can be skewed by the bike unit price
# Create a pivot table to analyze customer trends
df = ordersXcustomersXproducts.copy()
# First, use the groupby and aggregate function in pandas. 
customerTrends = df.groupby(['bikeshop.name', 'model', 'category1', 'category2', 'frame', 'Price Category']).agg({'quantity':'sum'})
# Then, create the pivot table using the built-in function in pandas
table = df.pivot_table(df,index=['model', 'category1', 'category2', 'frame', 'Price Category'],columns=['bikeshop.name'], aggfunc=np.sum, fill_value=0)
table

quantity  \
bikeshop.name                                                                        Albuquerque Cycles   
model                           category1 category2          frame    Price Category                      
Bad Habit 1                     Mountain  Trail              Aluminum High                            5   
Bad Habit 2                     Mountain  Trail              Aluminum Low                             2   
Beast of the East 1             Mountain  Trail              Aluminum High                            3   
Beast of the East 2             Mountain  Trail              Aluminum Low                             3   
Beast of the East 3             Mountain  Trail              Aluminum Low                             1   
CAAD Disc Ultegra               Road      Elite Road         Aluminum Low                             4   
CAAD12 105                      Road      Elite Road         Aluminum Low                             2   
CAAD12 Black Inc                Road      Elite Road         Aluminum High                            0   
CAAD12 Disc 105                 Road      Elite Road         Aluminum Low                             4   
CAAD12 Disc Dura Ace            Road      Elite Road         Aluminum High                           14   
CAAD12 Red                      Road      Elite Road         Aluminum High                            4   
CAAD12 Ultegra                  Road      Elite Road         Aluminum Low                             4   
CAAD8 105                       Road      Elite Road         Aluminum Low                             8   
CAAD8 Claris                    Road      Elite Road         Aluminum Low                             2   
CAAD8 Sora                      Road      Elite Road         Aluminum Low                             2   
CAAD8 Tiagra                    Road      Elite Road         Aluminum Low                             3   
Catalyst 1                      Mountain  Sport              Aluminum Low                             6   
Catalyst 2                      Mountain  Sport              Aluminum Low                             4   
Catalyst 3                      Mountain  Sport              Aluminum Low                             9   
Catalyst 4                      Mountain  Sport              Aluminum Low                             5   
F-Si 1                          Mountain  Cross Country Race Aluminum Low                             3   
F-Si 2                          Mountain  Cross Country Race Aluminum Low                             5   
F-Si 3                          Mountain  Cross Country Race Aluminum Low                             5   
F-Si Black Inc.                 Mountain  Cross Country Race Carbon   High                            0   
F-Si Carbon 2                   Mountain  Cross Country Race Carbon   High                            5   
F-Si Carbon 4                   Mountain  Cross Country Race Carbon   High                            3   
F-Si Hi-Mod 1                   Mountain  Cross Country Race Carbon   High                            1   
F-Si Hi-Mod Team                Mountain  Cross Country Race Carbon   High                            2   
Fat CAAD1                       Mountain  Fat Bike           Aluminum High                            4   
Fat CAAD2                       Mountain  Fat Bike           Aluminum Low                             5   
...                                                                                                 ...   
Supersix Evo Hi-Mod Utegra      Road      Elite Road         Carbon   High                            2   
Supersix Evo Red                Road      Elite Road         Carbon   High                            2   
Supersix Evo Tiagra             Road      Elite Road         Carbon   Low                             3   
Supersix Evo Ultegra 3          Road      Elite Road         Carbon   High                            2   
Supersix Evo Ultegra 4       

In [444]:
# Calulate the sum for each bike shop
shopsum = df.pivot_table(df,columns=['bikeshop.name'],aggfunc=np.sum)
shopsum

bikeshop.name  Albuquerque Cycles  Ann Arbor Speed  Austin Cruisers  \
quantity                      286              602              246   

bikeshop.name  Cincinnati Speed  Columbus Race Equipment  Dallas Cycles  \
quantity                    391                      394            234   

bikeshop.name  Denver Bike Shop  Detroit Cycles  Indianapolis Velocipedes  \
quantity                   2301             504                       319   

bikeshop.name  Ithaca Mountain Climbers      ...        \
quantity                           1264      ...         

bikeshop.name  Philadelphia Bike Shop  Phoenix Bi-peds  \
quantity                          245             1419   

bikeshop.name  Pittsburgh Mountain Machines  Portland Bi-peds  \
quantity                                377               644   

bikeshop.name  Providence Bi-peds  San Antonio Bike Shop  \
quantity                      542                    186   

bikeshop.name  San Francisco Cruisers  Seattle Race Equipment  Tampa 29ers  \
quantity                          374                     128          206   

bikeshop.name  Wichita Speed  
quantity                 507  

[1 rows x 30 columns]

In [445]:
# Scale the quantity data: Normalizing columns. (divide each value in the column by the sum of column)
for i in range(0, len(set(df['bikeshop.name']))):
    for j in range(0,len(table)):
        table.iloc[j,i] = table.iloc[j,i] / shopsum.iloc[0,i]
table    

quantity  \
bikeshop.name                                                                        Albuquerque Cycles   
model                           category1 category2          frame    Price Category                      
Bad Habit 1                     Mountain  Trail              Aluminum High                     0.017483   
Bad Habit 2                     Mountain  Trail              Aluminum Low                      0.006993   
Beast of the East 1             Mountain  Trail              Aluminum High                     0.010490   
Beast of the East 2             Mountain  Trail              Aluminum Low                      0.010490   
Beast of the East 3             Mountain  Trail              Aluminum Low                      0.003497   
CAAD Disc Ultegra               Road      Elite Road         Aluminum Low                      0.013986   
CAAD12 105                      Road      Elite Road         Aluminum Low                      0.006993   
CAAD12 Black Inc                Road      Elite Road         Aluminum High                     0.000000   
CAAD12 Disc 105                 Road      Elite Road         Aluminum Low                      0.013986   
CAAD12 Disc Dura Ace            Road      Elite Road         Aluminum High                     0.048951   
CAAD12 Red                      Road      Elite Road         Aluminum High                     0.013986   
CAAD12 Ultegra                  Road      Elite Road         Aluminum Low                      0.013986   
CAAD8 105                       Road      Elite Road         Aluminum Low                      0.027972   
CAAD8 Claris                    Road      Elite Road         Aluminum Low                      0.006993   
CAAD8 Sora                      Road      Elite Road         Aluminum Low                      0.006993   
CAAD8 Tiagra                    Road      Elite Road         Aluminum Low                      0.010490   
Catalyst 1                      Mountain  Sport              Aluminum Low                      0.020979   
Catalyst 2                      Mountain  Sport              Aluminum Low                      0.013986   
Catalyst 3                      Mountain  Sport              Aluminum Low                      0.031469   
Catalyst 4                      Mountain  Sport              Aluminum Low                      0.017483   
F-Si 1                          Mountain  Cross Country Race Aluminum Low                      0.010490   
F-Si 2                          Mountain  Cross Country Race Aluminum Low                      0.017483   
F-Si 3                          Mountain  Cross Country Race Aluminum Low                      0.017483   
F-Si Black Inc.                 Mountain  Cross Country Race Carbon   High                     0.000000   
F-Si Carbon 2                   Mountain  Cross Country Race Carbon   High                     0.017483   
F-Si Carbon 4                   Mountain  Cross Country Race Carbon   High                     0.010490   
F-Si Hi-Mod 1                   Mountain  Cross Country Race Carbon   High                     0.003497   
F-Si Hi-Mod Team                Mountain  Cross Country Race Carbon   High                     0.006993   
Fat CAAD1                       Mountain  Fat Bike           Aluminum High                     0.013986   
Fat CAAD2                       Mountain  Fat Bike           Aluminum Low                      0.017483   
...                                                                                                 ...   
Supersix Evo Hi-Mod Utegra      Road      Elite Road         Carbon   High                     0.006993   
Supersix Evo Red                Road      Elite Road         Carbon   High                     0.006993   
Supersix Evo Tiagra             Road      Elite Road         Carbon   Low                      0.010490   
Supersix Evo Ultegra 3          Road      Elite Road         Carbon   High                     0.006993   
Supersix Evo Ultegra 4       

## Step 2: Perform K-Means Clustering

### We believe that there are most likely to be at least four customer groups because of mountain bike vs road bike and premium vs affordable preferences. We also believe there could be more as some customers may not care about price but may still prefer a specific bike category. However, we’ll limit the clusters to eight as more is likely to overfit the segments. Plot silhouette results to find the best number of cluster is 5.


In [480]:
# Now, we are ready to perform the Kmeans analysis
number_of_clusters = 5

# Select appropriate columns in the dataset to perform the Kmeans attribute. Save these columns into a new data frame kmeansDat. 
# You will use this new dataframe to perform the Kmeans analysis
kmeansDat = table.copy()
# The data frame is transposed to have the customers as rows and models as columns.
kmeansDat=kmeansDat.transpose()
kmeans = KMeans(n_clusters=number_of_clusters) #Setup a KMeans model
model = kmeans.fit(X= kmeansDat) #Fit a model using selected data (kmeansDat). 

# You can see the lables by calling
labels = model.labels_

In [481]:
# In order to get the name of bike shop, we need to change the index of "bikeshop.name" to a column
kmeansDat.reset_index(level = 1,inplace = True)
kmeansDat

model                          bikeshop.name Bad Habit 1 Bad Habit 2  \
category1                                       Mountain    Mountain   
category2                                          Trail       Trail   
frame                                           Aluminum    Aluminum   
Price Category                                      High         Low   
quantity                  Albuquerque Cycles    0.017483    0.006993   
quantity                     Ann Arbor Speed    0.006645    0.009967   
quantity                     Austin Cruisers    0.008130    0.004065   
quantity                    Cincinnati Speed    0.005115    0.000000   
quantity             Columbus Race Equipment    0.010152    0.000000   
quantity                       Dallas Cycles    0.012821    0.017094   
quantity                    Denver Bike Shop    0.011734    0.013907   
quantity                      Detroit Cycles    0.009921    0.015873   
quantity            Indianapolis Velocipedes    0.006270    0.003135   
quantity            Ithaca Mountain Climbers    0.018196    0.011076   
quantity                   Kansas City 29ers    0.018150    0.015846   
quantity                    Las Vegas Cycles    0.001603    0.000000   
quantity                  Los Angeles Cycles    0.006289    0.009434   
quantity           Louisville Race Equipment    0.007595    0.000000   
quantity                Miami Race Equipment    0.004213    0.011236   
quantity               Minneapolis Bike Shop    0.018265    0.016743   
quantity                  Nashville Cruisers    0.008671    0.017341   
quantity             New Orleans Velocipedes    0.018478    0.002174   
quantity                     New York Cycles    0.007407    0.007407   
quantity        Oklahoma City Race Equipment    0.012987    0.009524   
quantity              Philadelphia Bike Shop    0.024490    0.004082   
quantity                     Phoenix Bi-peds    0.011276    0.019027   
quantity        Pittsburgh Mountain Machines    0.015915    0.002653   
quantity                    Portland Bi-peds    0.010870    0.010870   
quantity                  Providence Bi-peds    0.009225    0.023985   
quantity               San Antonio Bike Shop    0.021505    0.000000   
quantity              San Francisco Cruisers    0.002674    0.002674   
quantity              Seattle Race Equipment    0.015625    0.007812   
quantity                         Tampa 29ers    0.019417    0.000000   
quantity                       Wichita Speed    0.005917    0.000000   

model          Beast of the East 1 Beast of the East 2 Beast of the East 3  \
category1                 Mountain            Mountain            Mountain   
category2                    Trail               Trail               Trail   
frame                     Aluminum            Aluminum            Aluminum   
Price Category                High                 Low                 Low   
quantity                  0.010490            0.010490            0.003497   
quantity                  0.014950            0.009967            0.003322   
quantity                  0.008130            0.008130            0.000000   
quantity                  0.000000            0.000000            0.000000   
quantity                  0.000000            0.005076            0.002538   
quantity                  0.004274            0.004274            0.004274   
quantity                  0.018253            0.015211            0.016949   
quantity                  0.011905            0.005952            0.011905   
quantity                  0.009404            0.009404            0.000000   
quantity                  0.021361            0.018196            0.010285   
quantity                  0.018150            0.013829            0.018150   
quantity                  0.001603            0.000000            0.003205   
quantity                  0.025157            0.022013            0.000000   
quantity                  0.000000            0.005063            0.005063 

## Step 3: Analyze the result

In [482]:
# Now we get the clustering results, let's go through each cluster and analyze clustering results
clusters = {'Cluster1':"","Cluster2":"","Cluster3":"","Cluster4":"","Cluster5":""}

for index,label in enumerate(labels):
    # Figure out what customers are there in each cluster
    clusters['Cluster'+str(label+1)] = str(clusters.get('Cluster'+str(label+1))) + "\""+ kmeansDat.iloc[index,0] + "\" "

# Sort the dictionary
sorted_clusters = sorted(clusters.items(),key = lambda x : x[0])

# Print out customer names that are in each segment
print("Following clusters were found\n")
for tup in sorted_clusters:
    print(tup[0] + " : " + tup[1])

Following clusters were found

Cluster1 : "Ithaca Mountain Climbers" "Pittsburgh Mountain Machines" "Tampa 29ers" 
Cluster2 : "Albuquerque Cycles" "Dallas Cycles" "Denver Bike Shop" "Detroit Cycles" "Kansas City 29ers" "Los Angeles Cycles" "Minneapolis Bike Shop" "New York Cycles" "Phoenix Bi-peds" "Portland Bi-peds" "Providence Bi-peds" 
Cluster3 : "Ann Arbor Speed" "Austin Cruisers" "Indianapolis Velocipedes" "Miami Race Equipment" "Nashville Cruisers" "New Orleans Velocipedes" "Oklahoma City Race Equipment" "Seattle Race Equipment" 
Cluster4 : "Cincinnati Speed" "Columbus Race Equipment" "Las Vegas Cycles" "Louisville Race Equipment" "San Francisco Cruisers" "Wichita Speed" 
Cluster5 : "Philadelphia Bike Shop" "San Antonio Bike Shop" 


In [519]:
# Map clusters to the bike models
centers = model.cluster_centers_.transpose()
center = pd.DataFrame(centers, columns=['X1','X2','X3','X4','X5'])
# combine the table with bike information with the dataframe that stores the cluster centroid information
bikes = table.copy()
bikes.reset_index(level = 1,inplace = True)
bikes['X1'] = None
bikes['X2'] = None
bikes['X3'] = None
bikes['X4'] = None
bikes['X5'] = None
for i in range(0,len(bikes)):
    bikes.iloc[i,31] = center.iloc[i,0]
    bikes.iloc[i,32] = center.iloc[i,1]
    bikes.iloc[i,33] = center.iloc[i,2]
    bikes.iloc[i,34] = center.iloc[i,3]
    bikes.iloc[i,35] = center.iloc[i,4]

### Cluster 1

In [521]:
# Cluster 1: the top 10 models purchased are predominantly high-end and mountain. The all but one model has a carbon frame.
# Approach One: Using cluster centroids for feature inspection
cluster1bikes = bikes.sort_values(by=['X1'], ascending = False)
cluster1bikes.head(10)

category1  \
bikeshop.name                                                                         
model                          category2          frame    Price Category             
Scalpel-Si Carbon 3            Cross Country Race Carbon   High            Mountain   
Jekyll Carbon 4                Over Mountain      Carbon   High            Mountain   
Scalpel 29 Carbon Race         Cross Country Race Carbon   High            Mountain   
Trigger Carbon 3               Over Mountain      Carbon   High            Mountain   
Habit Carbon 2                 Trail              Carbon   High            Mountain   
Trigger Carbon 4               Over Mountain      Carbon   High            Mountain   
Catalyst 4                     Sport              Aluminum Low             Mountain   
Jekyll Carbon 2                Over Mountain      Carbon   High            Mountain   
Supersix Evo Hi-Mod Dura Ace 2 Elite Road         Carbon   High                Road   
Trigger Carbon 2               Over Mountain      Carbon   High            Mountain   

                                                                                    quantity  \
bikeshop.name                                                             Albuquerque Cycles   
model                          category2          frame    Price Category                      
Scalpel-Si Carbon 3            Cross Country Race Carbon   High                     0.003497   
Jekyll Carbon 4                Over Mountain      Carbon   High                     0.010490   
Scalpel 29 Carbon Race         Cross Country Race Carbon   High                     0.000000   
Trigger Carbon 3               Over Mountain      Carbon   High                     0.006993   
Habit Carbon 2                 Trail              Carbon   High                     0.003497   
Trigger Carbon 4               Over Mountain      Carbon   High                     0.006993   
Catalyst 4                     Sport              Aluminum Low                      0.017483   
Jekyll Carbon 2                Over Mountain      Carbon   High                     0.000000   
Supersix Evo Hi-Mod Dura Ace 2 Elite Road         Carbon   High                     0.017483   
Trigger Carbon 2               Over Mountain      Carbon   High                     0.003497   

                                                                                           \
bikeshop.name                                                             Ann Arbor Speed   
model                          category2          frame    Price Category                   
Scalpel-Si Carbon 3            Cross Country Race Carbon   High                  0.000000   
Jekyll Carbon 4                Over Mountain      Carbon   High                  0.014950   
Scalpel 29 Carbon Race         Cross Country Race Carbon   High                  0.003322   
Trigger Carbon 3               Over Mountain      Carbon   High                  0.000000   
Habit Carbon 2                 Trail              Carbon   High                  0.000000   
Trigger Carbon 4               Over Mountain      Carbon   High                  0.011628   
Catalyst 4                     Sport              Aluminum Low                   0.004983   
Jekyll Carbon 2                Over Mountain      Carbon   High                  0.001661   
Supersix Evo Hi-Mod Dura Ace 2 Elite Road         Carbon   High                  0.008306   
Trigger Carbon 2               Over Mountain      Carbon   High                  0.000000   

                                                                                           \
bikeshop.name                                                             Austin Cruisers   
model                          category2          frame    Price Category                   
Scalpel-Si Carbon 3            Cross Country Race Carbon   High                  0.000000   
Jekyll Carbon 4                Over Mountain      Carbon   High                  0.004065   
Scalpel 29 Carbon 

In [496]:
# Cluster 1: the top 10 models purchased are predominantly high-end and mountain. The all but one model has a carbon frame.
# Approach Two: Using sum of quantity of bike shops in this cluster
cluster1list = ["Ithaca Mountain Climbers","Pittsburgh Mountain Machines","Tampa 29ers" ]
cluster1data = kmeansDat[kmeansDat['bikeshop.name'].isin(cluster1list) ]
cluster1data = cluster1data.transpose()
cluster1data = cluster1data[1:]
cluster1data['Sum Quantity'] = 0
for i in range(0,len(cluster1data)) :
    for j in range(0, len(cluster1list)):
        cluster1data.iloc[i,len(cluster1list)] += cluster1data.iloc[i,j]
cluster1data.reset_index(level = 1,inplace = True)
cluster1models = cluster1data.sort_values(by = ['Sum Quantity'],ascending=False)
cluster1models.head(10)

category1  \
model                          category2          frame    Price Category             
Scalpel-Si Carbon 3            Cross Country Race Carbon   High            Mountain   
Jekyll Carbon 4                Over Mountain      Carbon   High            Mountain   
Scalpel 29 Carbon Race         Cross Country Race Carbon   High            Mountain   
Trigger Carbon 3               Over Mountain      Carbon   High            Mountain   
Habit Carbon 2                 Trail              Carbon   High            Mountain   
Trigger Carbon 4               Over Mountain      Carbon   High            Mountain   
Catalyst 4                     Sport              Aluminum Low             Mountain   
Jekyll Carbon 2                Over Mountain      Carbon   High            Mountain   
Supersix Evo Hi-Mod Dura Ace 2 Elite Road         Carbon   High                Road   
Trigger Carbon 2               Over Mountain      Carbon   High            Mountain   

                                                                             quantity  \
model                          category2          frame    Price Category               
Scalpel-Si Carbon 3            Cross Country Race Carbon   High             0.0237342   
Jekyll Carbon 4                Over Mountain      Carbon   High             0.0237342   
Scalpel 29 Carbon Race         Cross Country Race Carbon   High             0.0174051   
Trigger Carbon 3               Over Mountain      Carbon   High             0.0261076   
Habit Carbon 2                 Trail              Carbon   High             0.0197785   
Trigger Carbon 4               Over Mountain      Carbon   High             0.0150316   
Catalyst 4                     Sport              Aluminum Low             0.00553797   
Jekyll Carbon 2                Over Mountain      Carbon   High             0.0181962   
Supersix Evo Hi-Mod Dura Ace 2 Elite Road         Carbon   High             0.0110759   
Trigger Carbon 2               Over Mountain      Carbon   High              0.022943   

                                                                            quantity  \
model                          category2          frame    Price Category              
Scalpel-Si Carbon 3            Cross Country Race Carbon   High            0.0450928   
Jekyll Carbon 4                Over Mountain      Carbon   High            0.0185676   
Scalpel 29 Carbon Race         Cross Country Race Carbon   High            0.0424403   
Trigger Carbon 3               Over Mountain      Carbon   High            0.0371353   
Habit Carbon 2                 Trail              Carbon   High            0.0212202   
Trigger Carbon 4               Over Mountain      Carbon   High            0.0159151   
Catalyst 4                     Sport              Aluminum Low             0.0106101   
Jekyll Carbon 2                Over Mountain      Carbon   High            0.0159151   
Supersix Evo Hi-Mod Dura Ace 2 Elite Road         Carbon   High            0.0132626   
Trigger Carbon 2               Over Mountain      Carbon   High            0.0159151   

                                                                            quantity  \
model                          category2          frame    Price Category              
Scalpel-Si Carbon 3            Cross Country Race Carbon   High            0.0339806   
Jekyll Carbon 4                Over Mountain      Carbon   High            0.0485437   
Scalpel 29 Carbon Race         Cross Country Race Carbon   High            0.0242718   
Trigger Carbon 3               Over Mountain      Carbon   High            0.0145631   
Habit Carbon 2                 Trail              Carbon   High            0.0291262   
Trigger Carbon 4               Over Mountain      Carbon   High             0.038835   
Catalyst 4                     Sport              Aluminum Low             0.0485437   
Jekyll Carbon 2                Over Mountain      Carbon   High            0.0291262   
Supersix Evo Hi-Mod Dur

### Cluster 2

In [522]:
# Cluster 2: Is very similar to Cluster 5 with the majority of bikes in the low-end price range.
# Approach One: Using cluster centroids for feature inspection
cluster2bikes = bikes.sort_values(by=['X2'], ascending = False)
cluster2bikes.head(10)

category1  \
bikeshop.name                                                                  
model                   category2          frame    Price Category             
F-Si 2                  Cross Country Race Aluminum Low             Mountain   
Catalyst 3              Sport              Aluminum Low             Mountain   
F-Si Carbon 4           Cross Country Race Carbon   High            Mountain   
Trail 5                 Sport              Aluminum Low             Mountain   
CAAD8 Sora              Elite Road         Aluminum Low                 Road   
CAAD12 Disc 105         Elite Road         Aluminum Low                 Road   
CAAD8 105               Elite Road         Aluminum Low                 Road   
Habit 4                 Trail              Aluminum High            Mountain   
Synapse Carbon Disc 105 Endurance Road     Carbon   Low                 Road   
Trail 3                 Sport              Aluminum Low             Mountain   

                                                                             quantity  \
bikeshop.name                                                      Albuquerque Cycles   
model                   category2          frame    Price Category                      
F-Si 2                  Cross Country Race Aluminum Low                      0.017483   
Catalyst 3              Sport              Aluminum Low                      0.031469   
F-Si Carbon 4           Cross Country Race Carbon   High                     0.010490   
Trail 5                 Sport              Aluminum Low                      0.010490   
CAAD8 Sora              Elite Road         Aluminum Low                      0.006993   
CAAD12 Disc 105         Elite Road         Aluminum Low                      0.013986   
CAAD8 105               Elite Road         Aluminum Low                      0.027972   
Habit 4                 Trail              Aluminum High                     0.020979   
Synapse Carbon Disc 105 Endurance Road     Carbon   Low                      0.013986   
Trail 3                 Sport              Aluminum Low                      0.006993   

                                                                                    \
bikeshop.name                                                      Ann Arbor Speed   
model                   category2          frame    Price Category                   
F-Si 2                  Cross Country Race Aluminum Low                   0.003322   
Catalyst 3              Sport              Aluminum Low                   0.006645   
F-Si Carbon 4           Cross Country Race Carbon   High                  0.009967   
Trail 5                 Sport              Aluminum Low                   0.011628   
CAAD8 Sora              Elite Road         Aluminum Low                   0.018272   
CAAD12 Disc 105         Elite Road         Aluminum Low                   0.014950   
CAAD8 105               Elite Road         Aluminum Low                   0.023256   
Habit 4                 Trail              Aluminum High                  0.004983   
Synapse Carbon Disc 105 Endurance Road     Carbon   Low                   0.016611   
Trail 3                 Sport              Aluminum Low                   0.009967   

                                                                                    \
bikeshop.name                                                      Austin Cruisers   
model                   category2          frame    Price Category                   
F-Si 2                  Cross Country Race Aluminum Low                   0.008130   
Catalyst 3              Sport              Aluminum Low                   0.004065   
F-Si Carbon 4           Cross Country Race Carbon   High                  0.016260   
Trail 5                 Sport              Aluminum Low                   0.008130   
CAAD8 Sora              Elite Road         Aluminum Low                   0.012195   
CAAD12 Disc 105         Elite Road         Aluminum Low      

In [497]:
# Cluster 2: Is very similar to Cluster 5 with the majority of bikes in the low-end price range.
# Approach Two: Using sum of quantity of bike shops in this cluster
cluster2list = ["Albuquerque Cycles","Dallas Cycles","Denver Bike Shop","Detroit Cycles","Kansas City 29ers","Los Angeles Cycles","Minneapolis Bike Shop","New York Cycles","Phoenix Bi-peds","Portland Bi-peds","Providence Bi-peds"]
cluster2data = kmeansDat[kmeansDat['bikeshop.name'].isin(cluster2list) ]
cluster2data = cluster2data.transpose()
cluster2data = cluster2data[1:]
cluster2data['Sum Quantity'] = 0
for i in range(0,len(cluster2data)) :
    for j in range(0, len(cluster2list)):
        cluster2data.iloc[i,len(cluster2list)] += cluster2data.iloc[i,j]
cluster2data.reset_index(level = 1,inplace = True)
cluster2models = cluster2data.sort_values(by = ['Sum Quantity'],ascending=False)
cluster2models.head(10)

category1  \
model                   category2          frame    Price Category             
F-Si 2                  Cross Country Race Aluminum Low             Mountain   
Catalyst 3              Sport              Aluminum Low             Mountain   
F-Si Carbon 4           Cross Country Race Carbon   High            Mountain   
Trail 5                 Sport              Aluminum Low             Mountain   
CAAD8 Sora              Elite Road         Aluminum Low                 Road   
CAAD12 Disc 105         Elite Road         Aluminum Low                 Road   
CAAD8 105               Elite Road         Aluminum Low                 Road   
Habit 4                 Trail              Aluminum High            Mountain   
Synapse Carbon Disc 105 Endurance Road     Carbon   Low                 Road   
Trail 3                 Sport              Aluminum Low             Mountain   

                                                                      quantity  \
model                   category2          frame    Price Category               
F-Si 2                  Cross Country Race Aluminum Low              0.0174825   
Catalyst 3              Sport              Aluminum Low              0.0314685   
F-Si Carbon 4           Cross Country Race Carbon   High             0.0104895   
Trail 5                 Sport              Aluminum Low              0.0104895   
CAAD8 Sora              Elite Road         Aluminum Low             0.00699301   
CAAD12 Disc 105         Elite Road         Aluminum Low               0.013986   
CAAD8 105               Elite Road         Aluminum Low               0.027972   
Habit 4                 Trail              Aluminum High              0.020979   
Synapse Carbon Disc 105 Endurance Road     Carbon   Low               0.013986   
Trail 3                 Sport              Aluminum Low             0.00699301   

                                                                      quantity  \
model                   category2          frame    Price Category               
F-Si 2                  Cross Country Race Aluminum Low               0.025641   
Catalyst 3              Sport              Aluminum Low               0.042735   
F-Si Carbon 4           Cross Country Race Carbon   High             0.0128205   
Trail 5                 Sport              Aluminum Low              0.0299145   
CAAD8 Sora              Elite Road         Aluminum Low              0.0213675   
CAAD12 Disc 105         Elite Road         Aluminum Low             0.00854701   
CAAD8 105               Elite Road         Aluminum Low              0.0128205   
Habit 4                 Trail              Aluminum High             0.0128205   
Synapse Carbon Disc 105 Endurance Road     Carbon   Low               0.017094   
Trail 3                 Sport              Aluminum Low               0.025641   

                                                                      quantity  \
model                   category2          frame    Price Category               
F-Si 2                  Cross Country Race Aluminum Low              0.0182529   
Catalyst 3              Sport              Aluminum Low              0.0165146   
F-Si Carbon 4           Cross Country Race Carbon   High             0.0165146   
Trail 5                 Sport              Aluminum Low              0.0204259   
CAAD8 Sora              Elite Road         Aluminum Low              0.0108648   
CAAD12 Disc 105         Elite Road         Aluminum Low             0.00738809   
CAAD8 105               Elite Road         Aluminum Low             0.00869187   
Habit 4                 Trail              Aluminum High             0.0165146   
Synapse Carbon Disc 105 Endurance Road     Carbon   Low              0.0065189   
Trail 3                 Sport              Aluminum Low              0.0126032   

                                                                      quantity  \
model                   category2          frame    Price Category   

### Cluster 3

In [523]:
# Cluster 3: Tends to prefer road bikes that are low-end.
# Approach One: Using cluster centroids for feature inspection
cluster3bikes = bikes.sort_values(by=['X3'], ascending = False)
cluster3bikes.head(10)

category1  \
bikeshop.name                                                               
model                    category2      frame    Price Category             
Synapse Disc Tiagra      Endurance Road Aluminum Low                 Road   
CAAD12 Red               Elite Road     Aluminum High                Road   
Synapse Sora             Endurance Road Aluminum Low                 Road   
Slice Ultegra            Triathalon     Carbon   Low                 Road   
Supersix Evo Ultegra 3   Elite Road     Carbon   High                Road   
Synapse Disc 105         Endurance Road Aluminum Low                 Road   
Synapse Carbon Ultegra 4 Endurance Road Carbon   Low                 Road   
CAAD12 Ultegra           Elite Road     Aluminum Low                 Road   
Slice Ultegra D12        Triathalon     Carbon   High                Road   
Supersix Evo Ultegra 4   Elite Road     Carbon   Low                 Road   

                                                                          quantity  \
bikeshop.name                                                   Albuquerque Cycles   
model                    category2      frame    Price Category                      
Synapse Disc Tiagra      Endurance Road Aluminum Low                      0.017483   
CAAD12 Red               Elite Road     Aluminum High                     0.013986   
Synapse Sora             Endurance Road Aluminum Low                      0.000000   
Slice Ultegra            Triathalon     Carbon   Low                      0.010490   
Supersix Evo Ultegra 3   Elite Road     Carbon   High                     0.006993   
Synapse Disc 105         Endurance Road Aluminum Low                      0.027972   
Synapse Carbon Ultegra 4 Endurance Road Carbon   Low                      0.010490   
CAAD12 Ultegra           Elite Road     Aluminum Low                      0.013986   
Slice Ultegra D12        Triathalon     Carbon   High                     0.010490   
Supersix Evo Ultegra 4   Elite Road     Carbon   Low                      0.006993   

                                                                                 \
bikeshop.name                                                   Ann Arbor Speed   
model                    category2      frame    Price Category                   
Synapse Disc Tiagra      Endurance Road Aluminum Low                   0.013289   
CAAD12 Red               Elite Road     Aluminum High                  0.026578   
Synapse Sora             Endurance Road Aluminum Low                   0.009967   
Slice Ultegra            Triathalon     Carbon   Low                   0.013289   
Supersix Evo Ultegra 3   Elite Road     Carbon   High                  0.013289   
Synapse Disc 105         Endurance Road Aluminum Low                   0.011628   
Synapse Carbon Ultegra 4 Endurance Road Carbon   Low                   0.018272   
CAAD12 Ultegra           Elite Road     Aluminum Low                   0.013289   
Slice Ultegra D12        Triathalon     Carbon   High                  0.026578   
Supersix Evo Ultegra 4   Elite Road     Carbon   Low                   0.019934   

                                                                                 \
bikeshop.name                                                   Austin Cruisers   
model                    category2      frame    Price Category                   
Synapse Disc Tiagra      Endurance Road Aluminum Low                   0.028455   
CAAD12 Red               Elite Road     Aluminum High                  0.016260   
Synapse Sora             Endurance Road Aluminum Low                   0.020325   
Slice Ultegra            Triathalon     Carbon   Low                   0.040650   
Supersix Evo Ultegra 3   Elite Road     Carbon   High                  0.052846   
Synapse Disc 105         Endurance Road Aluminum Low                   0.020325   
Synapse Carbon Ultegra 4 Endurance Road Carbon   Low                   0.020325   
CAAD12 Ultegra           Elite Ro

In [491]:
# Cluster 3: Tends to prefer road bikes that are low-end.
# Approach Two: Using sum of quantity of bike shops in this cluster
cluster3list = ["Ann Arbor Speed","Austin Cruisers","Indianapolis Velocipedes","Miami Race Equipment","Nashville Cruisers","New Orleans Velocipedes","Oklahoma City Race Equipment","Seattle Race Equipment"]
cluster3data = kmeansDat[kmeansDat['bikeshop.name'].isin(cluster3list) ]
cluster3data = cluster3data.transpose()
cluster3data = cluster3data[1:]
cluster3data['Sum Quantity'] = 0
for i in range(0,len(cluster3data)) :
    for j in range(0, len(cluster3list)):
        cluster3data.iloc[i,len(cluster3list)] += cluster3data.iloc[i,j]
cluster3data.reset_index(level = 1,inplace = True)
cluster3models = cluster3data.sort_values(by = ['Sum Quantity'],ascending=False)
cluster3models.head(10)

category1  \
model                    category2      frame    Price Category             
Synapse Disc Tiagra      Endurance Road Aluminum Low                 Road   
CAAD12 Red               Elite Road     Aluminum High                Road   
Synapse Sora             Endurance Road Aluminum Low                 Road   
Slice Ultegra            Triathalon     Carbon   Low                 Road   
Supersix Evo Ultegra 3   Elite Road     Carbon   High                Road   
Synapse Disc 105         Endurance Road Aluminum Low                 Road   
Synapse Carbon Ultegra 4 Endurance Road Carbon   Low                 Road   
CAAD12 Ultegra           Elite Road     Aluminum Low                 Road   
Slice Ultegra D12        Triathalon     Carbon   High                Road   
Supersix Evo Ultegra 4   Elite Road     Carbon   Low                 Road   

                                                                   quantity  \
model                    category2      frame    Price Category               
Synapse Disc Tiagra      Endurance Road Aluminum Low               0.013289   
CAAD12 Red               Elite Road     Aluminum High             0.0265781   
Synapse Sora             Endurance Road Aluminum Low             0.00996678   
Slice Ultegra            Triathalon     Carbon   Low               0.013289   
Supersix Evo Ultegra 3   Elite Road     Carbon   High              0.013289   
Synapse Disc 105         Endurance Road Aluminum Low              0.0116279   
Synapse Carbon Ultegra 4 Endurance Road Carbon   Low              0.0182724   
CAAD12 Ultegra           Elite Road     Aluminum Low               0.013289   
Slice Ultegra D12        Triathalon     Carbon   High             0.0265781   
Supersix Evo Ultegra 4   Elite Road     Carbon   Low              0.0199336   

                                                                  quantity  \
model                    category2      frame    Price Category              
Synapse Disc Tiagra      Endurance Road Aluminum Low             0.0284553   
CAAD12 Red               Elite Road     Aluminum High            0.0162602   
Synapse Sora             Endurance Road Aluminum Low             0.0203252   
Slice Ultegra            Triathalon     Carbon   Low             0.0406504   
Supersix Evo Ultegra 3   Elite Road     Carbon   High            0.0528455   
Synapse Disc 105         Endurance Road Aluminum Low             0.0203252   
Synapse Carbon Ultegra 4 Endurance Road Carbon   Low             0.0203252   
CAAD12 Ultegra           Elite Road     Aluminum Low             0.0325203   
Slice Ultegra D12        Triathalon     Carbon   High            0.0121951   
Supersix Evo Ultegra 4   Elite Road     Carbon   Low             0.0365854   

                                                                   quantity  \
model                    category2      frame    Price Category               
Synapse Disc Tiagra      Endurance Road Aluminum Low              0.0407524   
CAAD12 Red               Elite Road     Aluminum High             0.0501567   
Synapse Sora             Endurance Road Aluminum Low             0.00626959   
Slice Ultegra            Triathalon     Carbon   Low             0.00940439   
Supersix Evo Ultegra 3   Elite Road     Carbon   High             0.0188088   
Synapse Disc 105         Endurance Road Aluminum Low              0.0282132   
Synapse Carbon Ultegra 4 Endurance Road Carbon   Low              0.0376176   
CAAD12 Ultegra           Elite Road     Aluminum Low              0.0219436   
Slice Ultegra D12        Triathalon     Carbon   High             0.0125392   
Supersix Evo Ultegra 4   Elite Road     Carbon   Low              0.0282132   

                                                                  quantity  \
model                    category2      frame    Price Category              
Synapse Disc Tiagra      Endurance Road Aluminum Low             0.0126404   
CAAD12 Red               Elite Road     Aluminum High            0.014

### Cluster 4

In [524]:
# Cluster 4: Tends to refer road bikes that are high-end.
# Approach One: Using cluster centroids for feature inspection
cluster4bikes = bikes.sort_values(by=['X4'], ascending = False)
cluster4bikes.head(10)

category1  \
bikeshop.name                                                                     
model                          category2      frame    Price Category             
Synapse Hi-Mod Disc Red        Endurance Road Carbon   High                Road   
Slice Hi-Mod Black Inc.        Triathalon     Carbon   High                Road   
Supersix Evo Hi-Mod Dura Ace 1 Elite Road     Carbon   High                Road   
Slice Hi-Mod Dura Ace D12      Triathalon     Carbon   High                Road   
Synapse Hi-Mod Dura Ace        Endurance Road Carbon   High                Road   
CAAD12 Red                     Elite Road     Aluminum High                Road   
Synapse Carbon Disc Ultegra    Endurance Road Carbon   High                Road   
Supersix Evo Ultegra 3         Elite Road     Carbon   High                Road   
Supersix Evo Hi-Mod Utegra     Elite Road     Carbon   High                Road   
Synapse Hi-Mod Disc Black Inc. Endurance Road Carbon   High                Road   

                                                                                quantity  \
bikeshop.name                                                         Albuquerque Cycles   
model                          category2      frame    Price Category                      
Synapse Hi-Mod Disc Red        Endurance Road Carbon   High                     0.024476   
Slice Hi-Mod Black Inc.        Triathalon     Carbon   High                     0.003497   
Supersix Evo Hi-Mod Dura Ace 1 Elite Road     Carbon   High                     0.000000   
Slice Hi-Mod Dura Ace D12      Triathalon     Carbon   High                     0.006993   
Synapse Hi-Mod Dura Ace        Endurance Road Carbon   High                     0.003497   
CAAD12 Red                     Elite Road     Aluminum High                     0.013986   
Synapse Carbon Disc Ultegra    Endurance Road Carbon   High                     0.006993   
Supersix Evo Ultegra 3         Elite Road     Carbon   High                     0.006993   
Supersix Evo Hi-Mod Utegra     Elite Road     Carbon   High                     0.006993   
Synapse Hi-Mod Disc Black Inc. Endurance Road Carbon   High                     0.000000   

                                                                                       \
bikeshop.name                                                         Ann Arbor Speed   
model                          category2      frame    Price Category                   
Synapse Hi-Mod Disc Red        Endurance Road Carbon   High                  0.008306   
Slice Hi-Mod Black Inc.        Triathalon     Carbon   High                  0.011628   
Supersix Evo Hi-Mod Dura Ace 1 Elite Road     Carbon   High                  0.013289   
Slice Hi-Mod Dura Ace D12      Triathalon     Carbon   High                  0.013289   
Synapse Hi-Mod Dura Ace        Endurance Road Carbon   High                  0.013289   
CAAD12 Red                     Elite Road     Aluminum High                  0.026578   
Synapse Carbon Disc Ultegra    Endurance Road Carbon   High                  0.008306   
Supersix Evo Ultegra 3         Elite Road     Carbon   High                  0.013289   
Supersix Evo Hi-Mod Utegra     Elite Road     Carbon   High                  0.008306   
Synapse Hi-Mod Disc Black Inc. Endurance Road Carbon   High                  0.004983   

                                                                                       \
bikeshop.name                                                         Austin Cruisers   
model                          category2      frame    Price Category                   
Synapse Hi-Mod Disc Red        Endurance Road Carbon   High                  0.016260   
Slice Hi-Mod Black Inc.        Triathalon     Carbon   High                  0.012195   
Supersix Evo Hi-Mod Dura Ace 1 Elite Road     Carbon   High                  0.004065   
Slice Hi-Mod Dura Ace D12      Triathalon     Carbon   High                  0.008130   
Synapse Hi-M

In [498]:
# Cluster 4: Tends to refer road bikes that are high-end.
# Approach Two: Using sum of quantity of bike shops in this cluster
cluster4list = ["Cincinnati Speed","Columbus Race Equipment","Las Vegas Cycles","Louisville Race Equipment","San Francisco Cruisers","Wichita Speed" ]
cluster4data = kmeansDat[kmeansDat['bikeshop.name'].isin(cluster4list) ]
cluster4data = cluster4data.transpose()
cluster4data = cluster4data[1:]
cluster4data['Sum Quantity'] = 0
for i in range(0,len(cluster4data)) :
    for j in range(0, len(cluster4list)):
        cluster4data.iloc[i,len(cluster4list)] += cluster4data.iloc[i,j]
cluster4data.reset_index(level = 1,inplace = True)
cluster4models = cluster4data.sort_values(by = ['Sum Quantity'],ascending=False)
cluster4models.head(10)

category1  \
model                          category2      frame    Price Category             
Synapse Hi-Mod Disc Red        Endurance Road Carbon   High                Road   
Slice Hi-Mod Black Inc.        Triathalon     Carbon   High                Road   
Supersix Evo Hi-Mod Dura Ace 1 Elite Road     Carbon   High                Road   
Slice Hi-Mod Dura Ace D12      Triathalon     Carbon   High                Road   
Synapse Hi-Mod Dura Ace        Endurance Road Carbon   High                Road   
CAAD12 Red                     Elite Road     Aluminum High                Road   
Synapse Carbon Disc Ultegra    Endurance Road Carbon   High                Road   
Supersix Evo Ultegra 3         Elite Road     Carbon   High                Road   
Supersix Evo Hi-Mod Utegra     Elite Road     Carbon   High                Road   
Synapse Hi-Mod Disc Black Inc. Endurance Road Carbon   High                Road   

                                                                         quantity  \
model                          category2      frame    Price Category               
Synapse Hi-Mod Disc Red        Endurance Road Carbon   High             0.0153453   
Slice Hi-Mod Black Inc.        Triathalon     Carbon   High             0.0179028   
Supersix Evo Hi-Mod Dura Ace 1 Elite Road     Carbon   High            0.00767263   
Slice Hi-Mod Dura Ace D12      Triathalon     Carbon   High             0.0204604   
Synapse Hi-Mod Dura Ace        Endurance Road Carbon   High             0.0179028   
CAAD12 Red                     Elite Road     Aluminum High              0.028133   
Synapse Carbon Disc Ultegra    Endurance Road Carbon   High             0.0332481   
Supersix Evo Ultegra 3         Elite Road     Carbon   High             0.0255754   
Supersix Evo Hi-Mod Utegra     Elite Road     Carbon   High             0.0230179   
Synapse Hi-Mod Disc Black Inc. Endurance Road Carbon   High             0.0255754   

                                                                        quantity  \
model                          category2      frame    Price Category              
Synapse Hi-Mod Disc Red        Endurance Road Carbon   High            0.0152284   
Slice Hi-Mod Black Inc.        Triathalon     Carbon   High            0.0228426   
Supersix Evo Hi-Mod Dura Ace 1 Elite Road     Carbon   High            0.0101523   
Slice Hi-Mod Dura Ace D12      Triathalon     Carbon   High            0.0126904   
Synapse Hi-Mod Dura Ace        Endurance Road Carbon   High            0.0380711   
CAAD12 Red                     Elite Road     Aluminum High            0.0177665   
Synapse Carbon Disc Ultegra    Endurance Road Carbon   High            0.0279188   
Supersix Evo Ultegra 3         Elite Road     Carbon   High            0.0253807   
Supersix Evo Hi-Mod Utegra     Elite Road     Carbon   High            0.0126904   
Synapse Hi-Mod Disc Black Inc. Endurance Road Carbon   High            0.0203046   

                                                                         quantity  \
model                          category2      frame    Price Category               
Synapse Hi-Mod Disc Red        Endurance Road Carbon   High             0.0400641   
Slice Hi-Mod Black Inc.        Triathalon     Carbon   High             0.0336538   
Supersix Evo Hi-Mod Dura Ace 1 Elite Road     Carbon   High              0.036859   
Slice Hi-Mod Dura Ace D12      Triathalon     Carbon   High              0.025641   
Synapse Hi-Mod Dura Ace        Endurance Road Carbon   High              0.025641   
CAAD12 Red                     Elite Road     Aluminum High            0.00641026   
Synapse Carbon Disc Ultegra    Endurance Road Carbon   High             0.0176282   
Supersix Evo Ultegra 3         Elite Road     Carbon   High             0.0144231   
Supersix Evo Hi-Mod Utegra     Elite Road     Carbon   High             0.0128205   
Synapse Hi-Mod Disc Black Inc. Endurance Road Carbon   High             0.0176282   

                      

### Cluster 5

In [525]:
# Cluster 5: the top models are all low-end/affordable models. 
# There’s a mix of road and mountain for the primariy category and a mix of frame material as well.
# Approach One: Using cluster centroids for feature inspection
cluster5bikes = bikes.sort_values(by=['X5'], ascending = False)
cluster5bikes.head(10)

category1  \
bikeshop.name                                                              
model               category2          frame    Price Category             
Slice Ultegra       Triathalon         Carbon   Low                 Road   
Trigger Carbon 4    Over Mountain      Carbon   High            Mountain   
CAAD12 105          Elite Road         Aluminum Low                 Road   
Beast of the East 3 Trail              Aluminum Low             Mountain   
Trail 1             Sport              Aluminum Low             Mountain   
Bad Habit 1         Trail              Aluminum High            Mountain   
F-Si Carbon 4       Cross Country Race Carbon   High            Mountain   
Synapse Disc 105    Endurance Road     Aluminum Low                 Road   
CAAD12 Disc 105     Elite Road         Aluminum Low                 Road   
Trail 2             Sport              Aluminum Low             Mountain   

                                                                         quantity  \
bikeshop.name                                                  Albuquerque Cycles   
model               category2          frame    Price Category                      
Slice Ultegra       Triathalon         Carbon   Low                      0.010490   
Trigger Carbon 4    Over Mountain      Carbon   High                     0.006993   
CAAD12 105          Elite Road         Aluminum Low                      0.006993   
Beast of the East 3 Trail              Aluminum Low                      0.003497   
Trail 1             Sport              Aluminum Low                      0.000000   
Bad Habit 1         Trail              Aluminum High                     0.017483   
F-Si Carbon 4       Cross Country Race Carbon   High                     0.010490   
Synapse Disc 105    Endurance Road     Aluminum Low                      0.027972   
CAAD12 Disc 105     Elite Road         Aluminum Low                      0.013986   
Trail 2             Sport              Aluminum Low                      0.003497   

                                                                                \
bikeshop.name                                                  Ann Arbor Speed   
model               category2          frame    Price Category                   
Slice Ultegra       Triathalon         Carbon   Low                   0.013289   
Trigger Carbon 4    Over Mountain      Carbon   High                  0.011628   
CAAD12 105          Elite Road         Aluminum Low                   0.014950   
Beast of the East 3 Trail              Aluminum Low                   0.003322   
Trail 1             Sport              Aluminum Low                   0.009967   
Bad Habit 1         Trail              Aluminum High                  0.006645   
F-Si Carbon 4       Cross Country Race Carbon   High                  0.009967   
Synapse Disc 105    Endurance Road     Aluminum Low                   0.011628   
CAAD12 Disc 105     Elite Road         Aluminum Low                   0.014950   
Trail 2             Sport              Aluminum Low                   0.014950   

                                                                                \
bikeshop.name                                                  Austin Cruisers   
model               category2          frame    Price Category                   
Slice Ultegra       Triathalon         Carbon   Low                   0.040650   
Trigger Carbon 4    Over Mountain      Carbon   High                  0.016260   
CAAD12 105          Elite Road         Aluminum Low                   0.020325   
Beast of the East 3 Trail              Aluminum Low                   0.000000   
Trail 1             Sport              Aluminum Low                   0.016260   
Bad Habit 1         Trail              Aluminum High                  0.008130   
F-Si Carbon 4       Cross Country Race Carbon   High                  0.016260   
Synapse Disc 105    Endurance Road     Aluminum Low                   0.020325   

In [499]:
# Cluster 5: the top models are all low-end/affordable models. 
# There’s a mix of road and mountain for the primariy category and a mix of frame material as well.
# Approach Two: Using sum of quantity of bike shops in this cluster
cluster5list = ["Philadelphia Bike Shop","San Antonio Bike Shop"]
cluster5data = kmeansDat[kmeansDat['bikeshop.name'].isin(cluster5list) ]
cluster5data = cluster5data.transpose()
cluster5data = cluster5data[1:]
cluster5data['Sum Quantity'] = 0
for i in range(0,len(cluster5data)) :
    for j in range(0, len(cluster5list)):
        cluster5data.iloc[i,len(cluster5list)] += cluster5data.iloc[i,j]
cluster5data.reset_index(level = 1,inplace = True)
cluster5models = cluster5data.sort_values(by = ['Sum Quantity'],ascending=False)
cluster5models.head(10)

category1  \
model               category2          frame    Price Category             
Slice Ultegra       Triathalon         Carbon   Low                 Road   
Trigger Carbon 4    Over Mountain      Carbon   High            Mountain   
CAAD12 105          Elite Road         Aluminum Low                 Road   
Beast of the East 3 Trail              Aluminum Low             Mountain   
Trail 1             Sport              Aluminum Low             Mountain   
Bad Habit 1         Trail              Aluminum High            Mountain   
F-Si Carbon 4       Cross Country Race Carbon   High            Mountain   
Synapse Disc 105    Endurance Road     Aluminum Low                 Road   
CAAD12 Disc 105     Elite Road         Aluminum Low                 Road   
Trail 2             Sport              Aluminum Low             Mountain   

                                                                 quantity  \
model               category2          frame    Price Category              
Slice Ultegra       Triathalon         Carbon   Low             0.0571429   
Trigger Carbon 4    Over Mountain      Carbon   High            0.0285714   
CAAD12 105          Elite Road         Aluminum Low             0.0326531   
Beast of the East 3 Trail              Aluminum Low             0.0204082   
Trail 1             Sport              Aluminum Low             0.0122449   
Bad Habit 1         Trail              Aluminum High            0.0244898   
F-Si Carbon 4       Cross Country Race Carbon   High             0.044898   
Synapse Disc 105    Endurance Road     Aluminum Low             0.0204082   
CAAD12 Disc 105     Elite Road         Aluminum Low             0.0204082   
Trail 2             Sport              Aluminum Low             0.0244898   

                                                                 quantity  \
model               category2          frame    Price Category              
Slice Ultegra       Triathalon         Carbon   Low             0.0537634   
Trigger Carbon 4    Over Mountain      Carbon   High            0.0322581   
CAAD12 105          Elite Road         Aluminum Low             0.0215054   
Beast of the East 3 Trail              Aluminum Low             0.0322581   
Trail 1             Sport              Aluminum Low             0.0376344   
Bad Habit 1         Trail              Aluminum High            0.0215054   
F-Si Carbon 4       Cross Country Race Carbon   High                    0   
Synapse Disc 105    Endurance Road     Aluminum Low             0.0215054   
CAAD12 Disc 105     Elite Road         Aluminum Low             0.0215054   
Trail 2             Sport              Aluminum Low              0.016129   

                                                                Sum Quantity  
model               category2          frame    Price Category                
Slice Ultegra       Triathalon         Carbon   Low                 0.110906  
Trigger Carbon 4    Over Mountain      Carbon   High                0.060829  
CAAD12 105          Elite Road         Aluminum Low                 0.054158  
Beast of the East 3 Trail              Aluminum Low                 0.052666  
Trail 1             Sport              Aluminum Low                 0.049879  
Bad Habit 1         Trail              Aluminum High                0.045995  
F-Si Carbon 4       Cross Country Race Carbon   High                0.044898  
Synapse Disc 105    Endurance Road     Aluminum Low                 0.041914  
CAAD12 Disc 105     Elite Road         Aluminum Low                 0.041914  
Trail 2             Sport              Aluminum Low                 0.040619

## Cluster Feature Summary

### Cluster1 : "Ithaca Mountain Climbers" "Pittsburgh Mountain Machines" "Tampa 29ers" 
Features: the top 10 models purchased are predominantly high-end and mountain. The all but one model has a carbon frame.


### Cluster2 : "Albuquerque Cycles" "Dallas Cycles" "Denver Bike Shop" "Detroit Cycles" "Kansas City 29ers" "Los Angeles Cycles" "Minneapolis Bike Shop" "New York Cycles" "Phoenix Bi-peds" "Portland Bi-peds" "Providence Bi-peds" 

Features: is very similar to Cluster 5 with the majority of bikes in the low-end price range.


### Cluster3 : "Ann Arbor Speed" "Austin Cruisers" "Indianapolis Velocipedes" "Miami Race Equipment" "Nashville Cruisers" "New Orleans Velocipedes" "Oklahoma City Race Equipment" "Seattle Race Equipment" 

Features: Tends to prefer road bikes that are low-end.


### Cluster4 : "Cincinnati Speed" "Columbus Race Equipment" "Las Vegas Cycles" "Louisville Race Equipment" "San Francisco Cruisers" "Wichita Speed" 

Features: Tends to refer road bikes that are high-end.


### Cluster5 : "Philadelphia Bike Shop" "San Antonio Bike Shop" 

Features: the top models are all low-end/affordable models. There’s a mix of road and mountain for the primariy category and a mix of frame material as well.